In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 98% 307M/315M [00:02<00:00, 186MB/s]
100% 315M/315M [00:02<00:00, 156MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'apnea-ecg-database.zip',
 'rri-amplitudes.ipynb',
 'kaggle.json',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [59:40<00:00,  7.38s/it]

Testing...
x12: 100%|██████████| 527/527 [1:01:25<00:00,  6.99s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import  Dense,Flatten,MaxPooling2D,Conv2D,Permute,Reshape,LSTM,BatchNormalization,Bidirectional
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D,Conv2D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd
from keras.layers import GRU
from keras.layers import add, concatenate
from keras.layers import Reshape, Lambda

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy

from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from tensorflow.keras import optimizers
#from Hyperactive.hyperactive import RandomSearchOptimizer, MHoneyBadgerAlgorithm, HoneyBadgerAlgorithm
from keras.optimizers import adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA
from tensorflow.python.keras import optimizers
from keras.optimizers import adam
#from HBA import HBA
import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 44.2 MB/s eta 0:00:00


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7],position[8],position[9],position[10],position[11]
    n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(n7),int(n8),int(n9),int(n10),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,epochs)

#def create_model(weight=1e-3):
    model=Sequential()
    model.add(Reshape((450,2,2),input_shape=(900,1,2)))
    model.add(Conv2D(n1, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n2, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n6, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))
    model.add(Conv2D(n7, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(Conv2D(n8, kernel_size=(3,1), strides=1, padding="same", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(MaxPooling2D(pool_size=(2,1)))

    model.add(Permute((2,1,3)))
    model.add(Reshape((2,28*n8)))

    model.add(Bidirectional(LSTM(n9, return_sequences=True)))

    model.add(Flatten())
    model.add(Dense(n10, activation="relu"))

    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,epochs

    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    #x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    #x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    #x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,32,32,64,64,120,120,256,256,64,21,100], # Lower bound of our parameters
    'ub':[0.001,256,256,500,500,700,700,900,900,256,64,100], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4, 1, 1,1,1,1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalHBA(epoch, pop_size)
  #t1 = time.time()

  #Optimizer.fit(x_train, y_train)
 # t2 = time.time()

  #print("time: {}".format(t2-t1))



#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.001 180 49 209 497 668 171 516 573 87 41 100


INFO:mealpy.swarm_based.HBA.OriginalHBA:Solving 12-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. ].


0.001 103 154 487 357 531 217 259 614 242 30 100
0.001 241 216 293 167 536 340 665 339 203 42 100
0.001 103 172 468 243 552 206 827 550 76 29 100
0.001 206 44 424 442 408 336 425 344 220 60 100
0.001 32 60 79 215 520 387 269 451 136 39 100
0.001 33 153 187 330 425 532 892 842 86 42 100
0.001 118 201 67 71 571 149 329 863 237 36 100
0.001 174 74 225 90 497 459 325 308 141 57 100
0.001 148 223 426 118 131 181 381 796 97 57 100
0.001 79 110 304 419 486 268 780 709 176 27 100
0.001 157 167 240 134 167 562 642 576 107 24 100
0.001 143 227 304 310 579 542 873 509 154 41 100
0.001 167 90 389 250 292 401 650 308 144 52 100
0.001 243 33 330 402 199 139 324 287 218 62 100
0.001 120 46 445 495 572 685 452 291 84 60 100
0.001 99 142 475 402 573 520 658 740 120 26 100
0.001 138 229 139 158 617 207 851 566 102 52 100
0.001 94 180 481 212 192 391 321 337 215 45 100
0.001 184 196 369 132 315 506 481 521 101 32 100
0.001 103 205 328 311 622 311 478 706 173 21 100
0.001 67 145 160 328 569 305 402 473 22

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 1, Current best: 1877.0004, Global best: 1877.0004, Runtime: 64.77627 seconds


0.001 36 64 66 145 301 262 256 299 110 30 100
0.001 39 79 72 178 412 367 264 341 91 34 100
0.001 42 140 66 169 257 334 285 337 97 31 100
0.001 56 112 64 181 280 299 352 353 89 33 100
0.001 34 74 65 169 303 272 272 318 122 25 100
0.001 43 120 64 147 352 320 256 256 91 31 100
0.001 46 132 80 227 477 277 377 296 99 28 100
0.001 133 42 204 124 162 151 298 349 96 21 100
0.001 39 76 64 145 576 269 256 345 100 24 100
0.001 32 66 64 64 372 152 256 347 100 38 100
0.001 45 123 69 178 139 354 305 279 64 29 100
0.001 35 39 80 185 265 349 256 261 69 25 100
0.001 32 113 78 207 430 394 256 351 113 37 100
0.001 137 32 253 64 120 297 280 256 102 21 100
0.001 40 61 64 108 249 345 272 359 73 35 100
0.001 44 97 69 132 372 339 305 370 64 22 100
0.001 50 51 64 179 343 138 355 302 114 28 100
0.001 32 109 64 145 326 315 353 354 82 27 100
0.001 42 93 64 167 255 251 273 343 64 28 100
0.001 43 87 64 166 307 332 267 351 64 31 100
0.001 46 91 64 182 394 322 280 352 110 31 100
0.001 41 93 64 111 273 251 284 341 94 

...

0.001 45 32 90 64 120 168 360 256 64 21 100
0.001 256 32 500 500 700 700 256 900 256 64 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 121 120 256 259 64 22 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 500 64 120 700 256 256 64 64 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 22 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 256 500 500 700 700 256 256 256 64 100
0.001 45 45 64 64 120 169 256 361 64 30 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 22 100
0.001 39 39 64 78 147 147 256 315 78 22 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 1

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 99, Current best: 1129.0004, Global best: 1129.0004, Runtime: 52.81944 seconds


0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 256 64 500 120 700 900 900 64 64 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 22 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 191 191 382 382 700 120 900 900 64 21 100
0.001 32 32 64 76 143 120 305 305 76 21 100
0.001 40 40 64 81 120 152 256 325 81 27 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 256 256 500 64 700 700 900 900 256 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 32 32 64 64 120 120 256 256 64 21 100
0.001 36 36 64 64 120 120 294 294 64 21 100
0.001 256 32 500 64 700 700 900 900 64 21 100
0.001 45 45 64 64 120 120 365 365 64 21 100
0.001 32 32 64 64 12

INFO:mealpy.swarm_based.HBA.OriginalHBA:>Problem: P, Epoch: 100, Current best: 1129.0004, Global best: 1129.0004, Runtime: 53.94897 seconds


0.001 32 32 64 64 120 120 256 256 64 21 100
Solution: [1.00000000e-03 3.20000000e+01 3.20000000e+01 6.40000000e+01
 6.40000000e+01 1.20000000e+02 1.20000000e+02 2.56000000e+02
 2.56000000e+02 6.40000000e+01 2.17205438e+01 1.00000000e+02], Fitness: (0.001, 32, 32, 64, 64, 120, 120, 256, 256, 64, 21, 100)
